In [1]:
"""
Copyright (c) Facebook, Inc. and its affiliates.

This source code is licensed under the MIT license found in the
LICENSE file in the root directory of this source tree.
"""

import os
import glob
import errno
import signal
import torch
import argparse
import warnings
import functools
import itertools
import numpy as np
import pandas as pd
from p_tqdm import p_map
import cloudpickle as pickle

from scipy.spatial.distance import pdist
from scipy.spatial.distance import cdist
from scipy.stats import wasserstein_distance

from collections import Counter
from pymatgen.core.structure import Structure
from pymatgen.core.composition import Composition
from pymatgen.core.lattice import Lattice
from matminer.featurizers.site.fingerprint import CrystalNNFingerprint
from matminer.featurizers.composition.composite import ElementProperty
import smact
from smact.screening import pauling_test

from eval_util import (
    chemical_symbols, 
    StandardScaler, 
    CompScalerMeans, 
    CompScalerStds
)


/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Much of the below code is taken without modification from the original
# CDVAE repo (https://github.com/txie-93/cdvae).
# In some cases, the code has been modified to work with the structure of
# our codebase, but the logic is the same.

COV_Cutoffs = {
    'mp20': {'struc': 0.4, 'comp': 10.},
    'carbon': {'struc': 0.2, 'comp': 4.},
    'perovskite': {'struc': 0.2, 'comp': 4},
}

NOVELTY_Cutoffs = {
    'mp20': {'struc': 0.1, 'comp': 2.},
}

CompScaler = StandardScaler(
    means=np.array(CompScalerMeans),
    stds=np.array(CompScalerStds),
    replace_nan_token=0.)

# CrystalNNFP = CrystalNNFingerprint.from_preset("ops")
CompFP = ElementProperty.from_preset('magpie')

class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wrapper

    return decorator

@timeout(5)
def timeout_featurize(structure, i):
    return CrystalNNFingerprint.from_preset("ops").featurize(structure, i)

def load_data(file_path):
    if file_path[-3:] == 'npy':
        data = np.load(file_path, allow_pickle=True).item()
        for k, v in data.items():
            if k == 'input_data_batch':
                for k1, v1 in data[k].items():
                    data[k][k1] = torch.from_numpy(v1)
            else:
                data[k] = torch.from_numpy(v).unsqueeze(0)
    else:
        data = torch.load(file_path)
    return data

def get_fp_pdist(fp_array):
    if isinstance(fp_array, list):
        fp_array = np.array(fp_array)
    fp_pdists = pdist(fp_array)
    return fp_pdists.mean()

def filter_fps(struc_fps, comp_fps):
    assert len(struc_fps) == len(comp_fps)

    filtered_struc_fps, filtered_comp_fps = [], []

    for struc_fp, comp_fp in zip(struc_fps, comp_fps):
        if struc_fp is not None and comp_fp is not None:
            filtered_struc_fps.append(struc_fp)
            filtered_comp_fps.append(comp_fp)
    return filtered_struc_fps, filtered_comp_fps

def compute_cov(
    crys, 
    gt_crys,
    struc_cutoff, 
    comp_cutoff, 
    num_gen_crystals=None
):
    struc_fps = [c.struct_fp for c in crys]
    comp_fps = [c.comp_fp for c in crys]
    gt_struc_fps = [c.struct_fp for c in gt_crys]
    gt_comp_fps = [c.comp_fp for c in gt_crys]

    assert len(struc_fps) == len(comp_fps)
    assert len(gt_struc_fps) == len(gt_comp_fps)

    # Use number of crystal before filtering to compute COV
    if num_gen_crystals is None:
        num_gen_crystals = len(struc_fps)

    struc_fps, comp_fps = filter_fps(struc_fps, comp_fps)
    gt_struc_fps, gt_comp_fps = filter_fps(gt_struc_fps, gt_comp_fps)

    comp_fps = CompScaler.transform(comp_fps)
    gt_comp_fps = CompScaler.transform(gt_comp_fps)

    struc_fps = np.array(struc_fps)
    gt_struc_fps = np.array(gt_struc_fps)
    comp_fps = np.array(comp_fps)
    gt_comp_fps = np.array(gt_comp_fps)

    struc_pdist = cdist(struc_fps, gt_struc_fps)
    comp_pdist = cdist(comp_fps, gt_comp_fps)

    struc_recall_dist = struc_pdist.min(axis=0)
    struc_precision_dist = struc_pdist.min(axis=1)
    comp_recall_dist = comp_pdist.min(axis=0)
    comp_precision_dist = comp_pdist.min(axis=1)

    cov_recall = np.mean(np.logical_and(
        struc_recall_dist <= struc_cutoff,
        comp_recall_dist <= comp_cutoff))
    cov_precision = np.mean(np.logical_and(
        struc_precision_dist <= struc_cutoff,
        comp_precision_dist <= comp_cutoff))# / num_gen_crystals

    metrics_dict = {
        'cov_recall': cov_recall,
        'cov_precision': cov_precision,
        'amsd_recall': np.mean(struc_recall_dist),
        'amsd_precision': np.mean(struc_precision_dist),
        'amcd_recall': np.mean(comp_recall_dist),
        'amcd_precision': np.mean(comp_precision_dist),
    }

    combined_dist_dict = {
        'struc_recall_dist': struc_recall_dist.tolist(),
        'struc_precision_dist': struc_precision_dist.tolist(),
        'comp_recall_dist': comp_recall_dist.tolist(),
        'comp_precision_dist': comp_precision_dist.tolist(),
    }

    return metrics_dict, combined_dist_dict

def compute_novelty(
    crys, 
    gt_crys,
    struc_cutoff, 
    comp_cutoff,
    num_gen_crystals=None
):
    struc_fps = [c.struct_fp for c in crys]
    comp_fps = [c.comp_fp for c in crys]
    gt_struc_fps = [c.struct_fp for c in gt_crys]
    gt_comp_fps = [c.comp_fp for c in gt_crys]

    assert len(struc_fps) == len(comp_fps)
    assert len(gt_struc_fps) == len(gt_comp_fps)

    # Use number of crystal before filtering to compute COV
    if num_gen_crystals is None:
        num_gen_crystals = len(struc_fps)

    struc_fps, comp_fps = filter_fps(struc_fps, comp_fps)
    gt_struc_fps, gt_comp_fps = filter_fps(gt_struc_fps, gt_comp_fps)

    comp_fps = CompScaler.transform(comp_fps)
    gt_comp_fps = CompScaler.transform(gt_comp_fps)

    struc_fps = np.array(struc_fps)
    gt_struc_fps = np.array(gt_struc_fps)
    comp_fps = np.array(comp_fps)
    gt_comp_fps = np.array(gt_comp_fps)

    struc_pdist = cdist(struc_fps, gt_struc_fps)
    comp_pdist = cdist(comp_fps, gt_comp_fps)

    struc_precision_dist = struc_pdist.min(axis=1)
    comp_precision_dist = comp_pdist.min(axis=1)

    struc_novelty = np.mean(struc_precision_dist > struc_cutoff)
    comp_novelty = np.mean(comp_precision_dist > comp_cutoff)

    novelty = np.mean(np.logical_or(
        struc_precision_dist > struc_cutoff,
        comp_precision_dist > comp_cutoff))

    metrics_dict = {
        'struc_novelty': struc_novelty,
        'comp_novelty': comp_novelty,
        'novelty': novelty,
    }

    return metrics_dict

class CDVAEGenEval(object):

    def __init__(
        self, 
        pred_crys, 
        gt_cov_crys, 
        gt_novelty_crys,
        n_samples=1000, 
        eval_model_name=None
    ):
        self.crys = pred_crys
        self.gt_cov_crys = gt_cov_crys
        self.gt_novelty_crys = gt_novelty_crys
        self.n_samples = n_samples
        self.eval_model_name = eval_model_name

        valid_crys = [c for c in pred_crys if c.valid]
        if len(valid_crys) >= n_samples:
            sampled_indices = np.random.choice(
                len(valid_crys), n_samples, replace=False)
            self.valid_samples = [valid_crys[i] for i in sampled_indices]
        else:
            raise Exception(
                f'not enough valid crystals in the predicted set: {len(valid_crys)}/{n_samples}')

    def get_validity(self):
        comp_valid = np.array([c.comp_valid for c in self.crys]).mean()
        struct_valid = np.array([c.struct_valid for c in self.crys]).mean()
        valid = np.array([c.valid for c in self.crys]).mean()
        return {'comp_valid': comp_valid,
                'struct_valid': struct_valid,
                'valid': valid}

    def get_comp_diversity(self):
        comp_fps = [c.comp_fp for c in self.valid_samples]
        comp_fps = CompScaler.transform(comp_fps)
        comp_div = get_fp_pdist(comp_fps)
        return {'comp_div': comp_div}

    def get_struct_diversity(self):
        return {'struct_div': get_fp_pdist([c.struct_fp for c in self.valid_samples])}

    def get_density_wdist(self):
        pred_densities = [c.structure.density for c in self.valid_samples]
        gt_densities = [c.structure.density for c in self.gt_cov_crys]
        wdist_density = wasserstein_distance(pred_densities, gt_densities)
        return {'wdist_density': wdist_density}

    def get_num_elem_wdist(self):
        pred_nelems = [len(set(c.structure.species))
                       for c in self.valid_samples]
        gt_nelems = [len(set(c.structure.species)) for c in self.gt_cov_crys]
        wdist_num_elems = wasserstein_distance(pred_nelems, gt_nelems)
        return {'wdist_num_elems': wdist_num_elems}

    def get_coverage(self):
        cutoff_dict = COV_Cutoffs[self.eval_model_name]
        (cov_metrics_dict, combined_dist_dict) = compute_cov(
            self.crys, self.gt_cov_crys,
            struc_cutoff=cutoff_dict['struc'],
            comp_cutoff=cutoff_dict['comp'])
        return cov_metrics_dict

    def get_novelty(self):
        cutoff_dict = NOVELTY_Cutoffs[self.eval_model_name]
        novelty_metrics_dict = compute_novelty(
            self.crys, self.gt_novelty_crys,
            struc_cutoff=cutoff_dict['struc'],
            comp_cutoff=cutoff_dict['comp'])
        return novelty_metrics_dict

    def get_metrics(self):
        metrics = {}
        metrics.update(self.get_validity())
        metrics.update(self.get_comp_diversity())
        metrics.update(self.get_struct_diversity())
        metrics.update(self.get_density_wdist())
        metrics.update(self.get_num_elem_wdist())
        metrics.update(self.get_coverage())
        metrics.update(self.get_novelty())
        return metrics


def smact_validity(comp, count,
                   use_pauling_test=True,
                   include_alloys=True):
    elem_symbols = tuple([chemical_symbols[elem] for elem in comp])
    space = smact.element_dictionary(elem_symbols)
    smact_elems = [e[1] for e in space.items()]
    electronegs = [e.pauling_eneg for e in smact_elems]
    ox_combos = [e.oxidation_states for e in smact_elems]
    if len(set(elem_symbols)) == 1:
        return True
    if include_alloys:
        is_metal_list = [elem_s in smact.metals for elem_s in elem_symbols]
        if all(is_metal_list):
            return True

    threshold = np.max(count)
    compositions = []
    for ox_states in itertools.product(*ox_combos):
        stoichs = [(c,) for c in count]
        # Test for charge balance
        cn_e, cn_r = smact.neutral_ratios(
            ox_states, stoichs=stoichs, threshold=threshold)
        # Electronegativity test
        if cn_e:
            if use_pauling_test:
                try:
                    electroneg_OK = pauling_test(ox_states, electronegs)
                except TypeError:
                    # if no electronegativity data, assume it is okay
                    electroneg_OK = True
            else:
                electroneg_OK = True
            if electroneg_OK:
                for ratio in cn_r:
                    compositions.append(
                        tuple([elem_symbols, ox_states, ratio]))
    compositions = [(i[0], i[2]) for i in compositions]
    compositions = list(set(compositions))
    if len(compositions) > 0:
        return True
    else:
        return False

def structure_validity(crystal, cutoff=0.5):
    dist_mat = crystal.distance_matrix
    # Pad diagonal with a large number
    dist_mat = dist_mat + np.diag(
        np.ones(dist_mat.shape[0]) * (cutoff + 10.))
    if dist_mat.min() < cutoff or crystal.volume < 0.1:
        return False
    else:
        return True

class Crystal(object):

    def __init__(self, crys_array_dict):
        self.frac_coords = crys_array_dict['frac_coords']
        self.atom_types = crys_array_dict['atom_types']
        self.lengths = crys_array_dict['lengths']
        self.angles = crys_array_dict['angles']
        self.dict = crys_array_dict

        self.get_structure()
        self.get_composition()
        self.get_validity()

        if self.valid:
            self.get_fingerprints()
        else:
            self.comp_fp = None
            self.struct_fp = None

    def get_structure(self):
        if min(self.lengths.tolist()) < 0:
            self.constructed = False
            self.invalid_reason = 'non_positive_lattice'
        else:
            try:
                self.structure = Structure(
                    lattice=Lattice.from_parameters(
                        *(self.lengths.tolist() + self.angles.tolist())),
                    species=self.atom_types, coords=self.frac_coords, coords_are_cartesian=False)
                self.constructed = True
            except Exception:
                self.constructed = False
                self.invalid_reason = 'construction_raises_exception'
            if self.structure.volume < 0.1:
                self.constructed = False
                self.invalid_reason = 'unrealistically_small_lattice'

    def get_composition(self):
        elem_counter = Counter(self.atom_types)
        composition = [(elem, elem_counter[elem])
                       for elem in sorted(elem_counter.keys())]
        elems, counts = list(zip(*composition))
        counts = np.array(counts)
        counts = counts / np.gcd.reduce(counts)
        self.elems = elems
        self.comps = tuple(counts.astype('int').tolist())

    def get_validity(self):
        self.comp_valid = smact_validity(self.elems, self.comps)
        if self.constructed:
            self.struct_valid = structure_validity(self.structure)
        else:
            self.struct_valid = False
        self.valid = self.comp_valid and self.struct_valid

    def get_fingerprints(self):
        elem_counter = Counter(self.atom_types)
        comp = Composition(elem_counter)
        self.comp_fp = CompFP.featurize(comp)
        try:
            site_fps = [timeout_featurize(
                self.structure, i) for i in range(len(self.structure))]
        except Exception as e:
            print(e)
            # counts crystal as invalid if fingerprint cannot be constructed.
            self.valid = False
            self.comp_fp = None
            self.struct_fp = None
            return
        self.struct_fp = np.array(site_fps).mean(axis=0)


def cif_str_to_crystal(cif_str):
    try:
        structure = Structure.from_str(cif_str, fmt="cif")
        crystal = Crystal({
            "frac_coords": structure.frac_coords,
            "atom_types": [chemical_symbols.index(str(x)) for x in structure.species],
            "lengths": np.array(structure.lattice.parameters[:3]),
            "angles": np.array(structure.lattice.parameters[3:])
        })
    except Exception as e:
        print(e)
        # print(cif_str)
        return None
    
    return crystal




In [4]:
baseline_numbers = pd.DataFrame([
    {'method': 'Train', 'struct_valid': 1.0, 'comp_valid': 0.9113, 'cov_recall': 1.0, 'cov_precision': 1.0, 'wdist_density': 0.051, 'wdist_num_elems': 0.016},
    {'method': 'FTCP', 'struct_valid': 0.0155, 'comp_valid': 0.4837, 'cov_recall': 0.047, 'cov_precision': 0.0009, 'wdist_density': 23.71, 'wdist_num_elems': 0.736},
    {'method': 'GSchNet', 'struct_valid': 0.9965, 'comp_valid': 0.7596, 'cov_recall': 0.3833, 'cov_precision': 0.9957, 'wdist_density': 3.034, 'wdist_num_elems': 0.641},
    {'method': 'PGSchNet', 'struct_valid': 0.7751, 'comp_valid': 0.7640, 'cov_recall': 0.4193, 'cov_precision': 0.9974, 'wdist_density': 4.04, 'wdist_num_elems': 0.623},
    {'method': 'CDVAE', 'struct_valid': 1.0, 'comp_valid': 0.867, 'cov_recall': 0.9915, 'cov_precision': 0.9949, 'wdist_density': 0.688, 'wdist_num_elems': 1.432},
    {'method': 'LM-CH', 'struct_valid': 0.8481, 'comp_valid': 0.8355, 'cov_recall': 0.9925, 'cov_precision': 0.9789, 'wdist_density': 0.864, 'wdist_num_elems': 0.132},
    {'method': 'LM-AC', 'struct_valid': 0.9581, 'comp_valid': 0.8887, 'cov_recall': 0.996, 'cov_precision': 0.9855, 'wdist_density': 0.696, 'wdist_num_elems': 0.092},
])

# results_df_fn = "generative_model_results_17k_14k.csv"
results_df_fn = "generative_model_results_17k_14k_congen_9046.csv"



In [5]:
# model_name = 'llamat-7b'
# model_name = 'llamat-8b-cif-congen'
model_name = 'llamat-2-7b-cif-congen-9046'

In [ ]:
def parse_fn(gen_str):
    lines = [x for x in gen_str.split("\n") if len(x) > 0]
    lengths = [float(x) for x in lines[0].split(" ")]
    angles = [float(x) for x in lines[1].split(" ")]
    species = [x for x in lines[2::2]]
    coords = [[float(y) for y in x.split(" ")] for x in lines[3::2]]
    
    structure = Structure(
        lattice=Lattice.from_parameters(
            *(lengths + angles)),
        species=species,
        coords=coords, 
        coords_are_cartesian=False,
    )
    
    return structure.to(fmt="cif")

In [7]:
import pandas as pd
import re

def parse_fn(gen_str):
    """
    Parse generated structure string to extract lengths, angles, species, and coordinates.
    
    The function handles cases where:
    1. The structure may not start from the first line
    2. There may be text before the actual structure data
    3. The first line should contain three numbers (lengths)
    4. The second line should contain three numbers (angles)a
    5. Then pairs of lines: element followed by three fractional coordinates
    """
    # Remove quotes and clean the string
    gen_str = gen_str.strip().strip('"')
    lines = [x.strip() for x in gen_str.split("\n") if len(x.strip()) > 0]
    
    # Find the start of the actual structure data
    # Look for a line that contains three numbers (lengths)
    start_idx = -1
    for i, line in enumerate(lines):
        # Split by whitespace and check if we have at least 3 numeric values
        parts = line.split()
        if len(parts) >= 3:
            # Try to find 3 consecutive numeric values in the line
            numeric_parts = []
            for part in parts:
                try:
                    val = float(part)
                    numeric_parts.append(val)
                    if len(numeric_parts) == 3:
                        break
                except ValueError:
                    continue
            
            # Check if we found 3 positive numbers (likely lengths)
            if len(numeric_parts) == 3 and all(x > 0 for x in numeric_parts):
                start_idx = i
                break
    
    if start_idx == -1 or start_idx >= len(lines) - 1:
        # If we can't find a valid start, return empty structure
        return [], [], [], []
    
    try:
        # Extract lengths (first line after start)
        lengths_line = lines[start_idx]
        # Find the first 3 numeric values in the line
        lengths = []
        for part in lengths_line.split():
            try:
                val = float(part)
                lengths.append(val)
                if len(lengths) == 3:
                    break
            except ValueError:
                continue
        
        # Extract angles (second line after start)
        if start_idx + 1 < len(lines):
            angles_line = lines[start_idx + 1]
            # Find the first 3 numeric values in the line
            angles = []
            for part in angles_line.split():
                try:
                    val = float(part)
                    angles.append(val)
                    if len(angles) == 3:
                        break
                except ValueError:
                    continue
        else:
            angles = []
        
        # Extract species and coordinates (pairs starting from third line)
        species = []
        coords = []
        
        # Start from the third line after the start index
        data_start = start_idx + 2
        
        for i in range(data_start, len(lines), 2):
            if i < len(lines):
                # Element line
                element_line = lines[i].strip()
                # Clean the element (remove any non-alphabetic characters at the end)
                element = re.sub(r'[^A-Za-z]', '', element_line)
                if element:  # Only add if we have a valid element
                    species.append(element)
                else:
                    # If no valid element, skip this pair
                    continue
                
                # Coordinates line (next line)
                if i + 1 < len(lines):
                    coords_line = lines[i + 1].strip()
                    try:
                        coord_parts = coords_line.split()
                        if len(coord_parts) >= 3:
                            coord_values = [float(x) for x in coord_parts[:3]]
                            coords.append(coord_values)
                        else:
                            # If coordinates are malformed, add empty coordinates
                            coords.append([0.0, 0.0, 0.0])
                    except ValueError:
                        # If coordinates can't be parsed, add empty coordinates
                        coords.append([0.0, 0.0, 0.0])
                else:
                    # If no coordinates line, add empty coordinates
                    coords.append([0.0, 0.0, 0.0])

        structure = Structure(
        lattice=Lattice.from_parameters(
            *(lengths + angles)),
        species=species,
        coords=coords, 
        coords_are_cartesian=False,)
        return structure.to(fmt="cif")

    except:
        return ''
    
    # return structure.to(fmt="cif")
        
    #     return lengths, angles, species, coords
        
    # except (ValueError, IndexError) as e:
    #     # If parsing fails, return empty structure
    #     return [], [], [], []

# def process_csv_file(input_file, output_file):
#     """Process a CSV file and save parsed results."""
#     print(f"Processing {input_file}...")
#     df = pd.read_csv(input_file)
    
#     outputs = []
#     failed_parses = []
#     successful_parses = 0
    
#     for idx, gen_str in enumerate(df['gen_str']):
#         if pd.isna(gen_str):
#             outputs.append({
#                 'lengths': [],
#                 'angles': [],
#                 'species': [],
#                 'coords': []
#             })
#             failed_parses.append({
#                 'index': idx,
#                 'original_gen_str': gen_str,
#                 'reason': 'NaN input'
#             })
#             continue
            
#         lengths, angles, species, coords = parse_fn(str(gen_str))
        
#         # Check if we got valid data
#         if len(lengths) == 3 and len(angles) == 3 and len(species) > 0:
#             successful_parses += 1
#         else:
#             # Record failed parse
#             failed_parses.append({
#                 'index': idx,
#                 'original_gen_str': str(gen_str),
#                 'reason': f'Invalid structure: lengths={len(lengths)}, angles={len(angles)}, species={len(species)}'
#             })
        
#         outputs.append({
#             'lengths': lengths,
#             'angles': angles,
#             'species': species,
#             'coords': coords
#         })
        
#         if (idx + 1) % 1000 == 0:
#             print(f"Processed {idx + 1} structures...")
    
#     result_df = pd.DataFrame(outputs)
#     result_df.to_csv(output_file, index=False)
#     print(f"Saved {len(outputs)} parsed structures to {output_file}")
#     print(f"Successfully parsed {successful_parses} out of {len(outputs)} structures")
    
#     # Save failed parses
#     if failed_parses:
#         failed_df = pd.DataFrame(failed_parses)
#         failed_output_file = output_file.replace('.csv', '_failed.csv')
#         failed_df.to_csv(failed_output_file, index=False)
#         print(f"Saved {len(failed_parses)} failed structures to {failed_output_file}")
    
#     return result_df

# # Process both datasets
# if __name__ == "__main__":
#     # Process llamat2 dataset
#     try:
#         process_csv_file("llamat2_9046.csv", "llamat2_9046_parsed.csv")
#     except FileNotFoundError:
#         print("llamat2_9046.csv not found, skipping...")
    
#     # Process llamat3 dataset
#     try:
#         process_csv_file("llamat3_9046.csv", "llamat3_9046_parsed.csv")
#     except FileNotFoundError:
#         print("llamat3_9046.csv not found, skipping...")

In [42]:
# samples_path = "llamat3_14000_cif.csv"
# samples_path = 'llm_samples_17k_cif.csv'
# samples_path = 'llamat2_cif.csv'
# samples_path = 'llamat2_cif_adapter_cleaned.csv'
# samples_path = 'llamat3_cif_adapter_cleaned.csv'
samples_path = 'llamat2_9064_llamat2_1758654262.csv'
# llamat3_cif_adapter_cleaned

In [43]:
# pd.read_csv('llamat3_9064_llamat3_1758654262.csv').head(2)

In [9]:
dfres = pd.read_csv(samples_path)

In [11]:
from tqdm import tqdm

In [12]:
parsed = []
for gen_str in tqdm(dfres.gen_str):
    parsed.append(parse_fn(gen_str))

 95%|██████████████████████████████████████████████████████████████████████████▏   | 8602/9046 [00:08<00:00, 948.63it/s]/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/core/lattice.py:426: RuntimeWarning: divide by zero encountered in scalar divide
  val = (cos_alpha * cos_beta - cos_gamma) / (sin_alpha * sin_beta)
 99%|████████████████████████████████████████████████████████████████████████████▉ | 8930/9046 [00:08<00:00, 986.29it/s]/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 9046/9046 [00:09<00:00, 991.96it/s]


In [14]:
parsed[0]

"# generated using pymatgen\ndata_GaTe\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   4.20000000\n_cell_length_b   4.20000000\n_cell_length_c   6.70000000\n_cell_angle_alpha   90.00000000\n_cell_angle_beta   90.00000000\n_cell_angle_gamma   120.00000000\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   GaTe\n_chemical_formula_sum   'Ga2 Te2'\n_cell_volume   102.35381042\n_cell_formula_units_Z   2\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Ga  Ga0  1  0.37000000  0.85000000  0.89000000  1\n  Ga  Ga1  1  0.03000000  0.19000000  0.39000000  1\n  Te  Te2  1  0.37000000  0.85000000  0.51000000  1\n  Te  Te3  1  0.03000000  0.19000000  0.01000000  1\n"

In [17]:
if os.path.exists(results_df_fn):
    results_df = pd.read_csv(results_df_fn)
else:
    baseline_numbers.to_csv(results_df_fn, index=False)
    results_df = baseline_numbers

if model_name in results_df["method"].values:
    print(f"Skipping {args.model_name} because it already exists in {results_df_fn}")
    # return

csv_fns = [
    x for x in glob.glob(samples_path) 
        if len(open(x).readlines()) > 1 and 'm3gnet_relaxed_energy' not in x
]
if len(csv_fns) == 0:
    # return
    print(len(csv_fns))


In [18]:

csv_fns

['llamat2_9064_llamat2_1758654262.csv']

In [19]:
pred_cifs = []
for x in csv_fns:
    try:
        df = pd.read_csv(x)
        df['cif'] = df.gen_str.apply(lambda x: parse_fn(x))
        pred_cifs += list(df["cif"].dropna())
    except:
        pass

/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/core/lattice.py:426: RuntimeWarning: divide by zero encountered in scalar divide
  val = (cos_alpha * cos_beta - cos_gamma) / (sin_alpha * sin_beta)
/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(


In [41]:
pred_cifs = pred_cifs[::-1]
print(len(pred_cifs))

502


In [42]:
samples_path

'llamat3_cif_adapter_cleaned.csv'

In [20]:
# fname = f"pred_{samples_path[:-4]}.pkl";
fname = f"all_congen_pred_{samples_path[:-4]}.pkl";
# if 'pred_crys_17k_14k_cif.pkl' not in os.listdir():
if fname not in os.listdir():
    pred_crys = [x for x in p_map(cif_str_to_crystal, pred_cifs) if x is not None]
    with open(fname,'wb') as f:
        pickle.dump(pred_crys, f)
    f.close()
else:
    print(f'loading {fname}')
    with open(fname,'rb') as f:
        pred_crys = pickle.load(f)
    f.close()

  0%|                                                                                          | 0/9046 [00:00<?, ?it/s]

Invalid CIF file with no structures!
Invalid CIF file with no structures!Invalid CIF file with no structures!
Invalid CIF file with no structures!



/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4151: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4151: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4151: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4151: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the stru

Invalid CIF file with no structures!


  0%|                                                                                | 2/9046 [00:03<3:28:48,  1.39s/it]

...

 98%|█████████████████████████████████████████████████████████████████████████████▋ | 8900/9046 [16:08<00:11, 12.50it/s]

Invalid CIF file with no structures!
Invalid CIF file with no structures!
Invalid CIF file with no structures!
Invalid CIF file with no structures!
Invalid CIF file with no structures!


 98%|█████████████████████████████████████████████████████████████████████████████▊ | 8907/9046 [16:11<00:17,  7.99it/s]

Invalid CIF file with no structures!


100%|███████████████████████████████████████████████████████████████████████████████| 9046/9046 [16:17<00:00,  9.25it/s]


In [21]:
import cloudpickle as pickle
# with open(fname,'wb') as f:
#    pickle.dump(pred_crys, f)
# f.close()
from copy import copy
pred_crystal = copy(pred_crys)
# with open(fname, 'rb') as f:
#     pred_crystal = pickle.load(f)

In [22]:
# nsamps = 10000
nsamps = 9046
if len(pred_crystal) > nsamps:
    # random_idx = np.random.choice(len(pred_crystal), 10000)
    # pred_crys = [pred_crystal[x] for x in random_idx]
    pred_crys = pred_crystal[:nsamps]

In [23]:
len(pred_crys)

8274

In [24]:
# if len(pred_crys) > 10000:
#     random_idx = np.random.choice(len(pred_crys), 10000)
#     pred_crys = [pred_crys[x] for x in random_idx]

In [25]:
samples_path

'llamat2_9064_llamat2_1758654262.csv'

In [26]:
os.makedirs(samples_path[:-4], exist_ok=True)

In [27]:
from tqdm import tqdm

In [28]:
pwd

'/home/scai/phd/aiz218326/github/crystal-text-llm'

In [29]:
for i, cc in tqdm(enumerate(pred_crys)):
    cc.structure.to_file(f'./{samples_path[:-4]}/{i}.cif')

8167it [01:03, 139.09it/s]/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
8274it [01:04, 128.50it/s]


In [53]:
validss = []
os.makedirs(f"{samples_path[:-4]}-valids", exist_ok=True)
for idx, cc in enumerate(pred_crys):
    if len(validss)<10:
        if cc.struct_valid & cc.comp_valid:
            
            
            cc.structure.to_file(f'./{samples_path[:-4]}-valids/{idx}.cif')
            validss.append(cc)
            print(idx, f'./{samples_path[:-4]}-valids/{idx}.cif')


2 ./llamat3_cif_adapter_cleaned-valids/2.cif
10 ./llamat3_cif_adapter_cleaned-valids/10.cif
13 ./llamat3_cif_adapter_cleaned-valids/13.cif
17 ./llamat3_cif_adapter_cleaned-valids/17.cif
18 ./llamat3_cif_adapter_cleaned-valids/18.cif
21 ./llamat3_cif_adapter_cleaned-valids/21.cif
22 ./llamat3_cif_adapter_cleaned-valids/22.cif
23 ./llamat3_cif_adapter_cleaned-valids/23.cif
24 ./llamat3_cif_adapter_cleaned-valids/24.cif
25 ./llamat3_cif_adapter_cleaned-valids/25.cif


In [30]:
samples_path

'llamat2_9064_llamat2_1758654262.csv'

In [31]:
import ase

In [32]:
ase.io.read

<function ase.io.formats.read(filename: Union[str, pathlib.PurePath, IO], index: Any = None, format: Optional[str] = None, parallel: bool = True, do_not_split_by_at_sign: bool = False, **kwargs) -> Union[ase.atoms.Atoms, List[ase.atoms.Atoms]]>

In [34]:
# ase.io.read('checkme.cif')

In [35]:
test_cov_path = 'data/basic/test.csv'

In [36]:
gt_cov_cifs = pd.read_csv(test_cov_path)["cif"]

gt_cov_crys_fn = test_cov_path.replace(".csv", "_cached.pkl")

In [37]:
if not os.path.exists(gt_cov_crys_fn):
    gt_cov_crys = p_map(cif_str_to_crystal, gt_cov_cifs)
    pickle.dump(gt_cov_crys, open(gt_cov_crys_fn, "wb"))
    print('saved')
else:
    print('loading ',gt_cov_crys_fn)
    gt_cov_crys = pickle.load(open(gt_cov_crys_fn, "rb"))

  0%|                                                                                          | 0/9046 [00:00<?, ?it/s]/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/io/cif.py:1287: UserWarning: Issues encountered while parsing CIF: 16 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/io/cif.py:1287: UserWarning: Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/io/cif.py:1287: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issu

Timer expired


 29%|██████████████████████▌                                                        | 2588/9046 [04:42<13:30,  7.97it/s]/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/io/cif.py:1287: UserWarning: Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
 30%|███████████████████████▋                                                       | 2709/9046 [04:49<06:42, 15.73it/s]/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/io/cif.py:1287: UserWarning: Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
 30%|███████████████████████▊                                                       | 2722/9046 [0

Timer expired


 32%|█████████████████████████                                                      | 2871/9046 [05:10<17:30,  5.88it/s]/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/io/cif.py:1287: UserWarning: Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
 33%|█████████████████████████▋                                                     | 2946/9046 [05:15<07:53, 12.89it/s]

...

Timer expired


 99%|██████████████████████████████████████████████████████████████████████████████ | 8935/9046 [15:54<00:10, 10.51it/s]/home/scai/phd/aiz218326/anaconda3/envs/crystal-llm/lib/python3.10/site-packages/pymatgen/io/cif.py:1287: UserWarning: Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
 99%|██████████████████████████████████████████████████████████████████████████████▍| 8983/9046 [15:58<00:05, 11.24it/s]

Timer expired


100%|███████████████████████████████████████████████████████████████████████████████| 9046/9046 [16:01<00:00,  9.41it/s]


saved


In [38]:
test_novelty_path = 'data/basic/train.csv'
gt_novelty_cifs = pd.read_csv(test_novelty_path)["cif"]

In [44]:
gt_novelty_cifs = pd.read_csv(test_novelty_path)["cif"]

gt_novelty_crys_fn = test_novelty_path.replace(".csv", "_cached.pkl")

In [48]:
if not os.path.exists(gt_novelty_crys_fn):
    gt_novelty_crys = p_map(cif_str_to_crystal, gt_novelty_cifs)
    pickle.dump(gt_novelty_crys, open(gt_novelty_crys_fn, "wb"))
    print('saved')
else:
    print('loading')
    gt_novelty_crys = pickle.load(open(gt_novelty_crys_fn, "rb"))
    print('loaded from',gt_novelty_crys_fn)

loading
loaded from data/basic/train_cached.pkl


In [49]:
type(gt_novelty_crys)

list

In [50]:
valid_crys = [x for x in pred_crys if x.valid]

print("Number of pred crystals: ", len(pred_crys))
print("Number of valid crystals: ", len(valid_crys))
print("ratio", len(valid_crys)/len(pred_crys))

Number of pred crystals:  8274
Number of valid crystals:  7252
ratio 0.8764805414551607


In [65]:
994/1000, 868/994 #llamat-2-cif-congen

(0.994, 0.8732394366197183)

In [66]:
491/1000, 209/491 #llamat-3-cif-congen

(0.491, 0.4256619144602851)

In [25]:
9958/10000, 8650/10000, 8650/9958

(0.9958, 0.865, 0.8686483229564169)

In [51]:
metrics = CDVAEGenEval(
        pred_crys, 
        gt_cov_crys,
        gt_novelty_crys,
        n_samples=len(valid_crys), 
        eval_model_name='mp20'
    )#.get_metrics()

In [52]:
metrics.get_metrics()

{'comp_valid': 0.8954556441866086,
 'struct_valid': 0.9772782209330433,
 'valid': 0.8764805414551607,
 'comp_div': 15.520819779408347,
 'struct_div': 0.9002762982343132,
 'wdist_density': 0.17426589733296383,
 'wdist_num_elems': 0.01680809819371447,
 'cov_recall': 0.9938837920489296,
 'cov_precision': 0.9951737451737451,
 'amsd_recall': 0.11372955840352154,
 'amsd_precision': 0.133831247137884,
 'amcd_recall': 0.5117096469684149,
 'amcd_precision': 0.20458088840637634,
 'struc_novelty': 0.5124103695532267,
 'comp_novelty': 0.34624931053502483,
 'novelty': 0.6840871483728627}

In [60]:
metrics = {'comp_valid': 0.6505,
 'struct_valid': 0.7281,
 'valid': 0.5199,
 'comp_div': 15.049131375700867,
 'struct_div': 0.646087535174593,
 'wdist_density': 10.404570939360589,
 'wdist_num_elems': 0.12935201105231345,
 'cov_recall': 0.9593515903229768,
 'cov_precision': 0.9824966339680707,
 'amsd_recall': 0.19431147943564403,
 'amsd_precision': 0.21623487953435078,
 'amcd_recall': 4.174398269649543,
 'amcd_precision': 3.7243309433717013,
 'struc_novelty': 0.8726678207347567,
 'comp_novelty': 0.7357184073860358,
 'novelty': 0.9224850932871707}

In [61]:
metrics_llamat3 = {'comp_valid': 0.6028574617702869,
 'struct_valid': 0.6742940060274584,
 'valid': 0.45563120884027236,
 'comp_div': 14.813907952342776,
 'struct_div': 0.5931430120303295,
 'wdist_density': 12.35512616311076,
 'wdist_num_elems': 0.2607242443028679,
 'cov_recall': 0.9247206189365099,
 'cov_precision': 0.9936305732484076,
 'amsd_recall': 0.21367498510494704,
 'amsd_precision': 0.22137167569311592,
 'amcd_recall': 4.792127000566393,
 'amcd_precision': 3.9909140918141657,
 'struc_novelty': 0.9152376286134248,
 'comp_novelty': 0.7893189612934836,
 'novelty': 0.9468397844194023}

In [62]:
metrics_llamat2 = {'comp_valid': 0.8783,
 'struct_valid': 0.9951,
 'valid': 0.8755,
 'comp_div': 15.537416884939825,
 'struct_div': 0.9754777500902315,
 'wdist_density': 0.6225818111254449,
 'wdist_num_elems': 0.022798721630026456,
 'cov_recall': 0.9863686601989439,
 'cov_precision': 0.99565962307253,
 'amsd_recall': 0.13205219925068698,
 'amsd_precision': 0.11404735865757508,
 'amcd_recall': 3.237658132946266,
 'amcd_precision': 2.5953085459418874,
 'struc_novelty': 0.39771559109080523,
 'comp_novelty': 0.36070816676185036,
 'novelty': 0.5888063963449457}

In [75]:
metrics_old = {'comp_valid': 0.8726651938140189,
 'struct_valid': 0.9945772243422374,
 'valid': 0.8686483229564169,
 'comp_div': 15.435847558071819,
 'struct_div': 0.997068275992884,
 'wdist_density': 0.6633393952088423,
 'wdist_num_elems': 0.07941215035802868,
 'cov_recall': 0.9878423185558148,
 'cov_precision': 0.9922543352601156,
 'amsd_recall': 0.12996201483321324,
 'amsd_precision': 0.11859335672718942,
 'amcd_recall': 3.002275575248583,
 'amcd_precision': 2.6840815605625474,
 'struc_novelty': 0.41063583815028903,
 'comp_novelty': 0.41132947976878614,
 'novelty': 0.6338728323699422}
for key in metrics.keys():
    metrics[key] = round(metrics[key], 3);
    metrics_llamat2[key] = round(metrics_llamat2[key], 3);
    metrics_llamat3[key] = round(metrics_llamat3[key], 3);


In [63]:
metrics_llamat2_congen = {'comp_valid': 0.65,
 'struct_valid': 0.728,
 'valid': 0.52,
 'comp_div': 15.049,
 'struct_div': 0.646,
 'wdist_density': 10.405,
 'wdist_num_elems': 0.129,
 'cov_recall': 0.959,
 'cov_precision': 0.982,
 'amsd_recall': 0.194,
 'amsd_precision': 0.216,
 'amcd_recall': 4.174,
 'amcd_precision': 3.724,
 'struc_novelty': 0.873,
 'comp_novelty': 0.736,
 'novelty': 0.922,
 'method': 'llamat2-conditional'}

# {'comp_valid': 0.8782696177062375,
#  'struct_valid': 0.9929577464788732,
#  'valid': 0.8732394366197183,
#  'comp_div': 15.571199340292052,
#  'struct_div': 0.9869229138181784,
#  'wdist_density': 0.6994115001718545,
#  'wdist_num_elems': 0.07296959421940691,
#  'cov_recall': 0.9292643988701953,
#  'cov_precision': 0.9976958525345622,
#  'amsd_recall': 0.1893616040611489,
#  'amsd_precision': 0.11334040556446105,
#  'amcd_recall': 5.15971819523058,
#  'amcd_precision': 2.630082032588238,
#  'struc_novelty': 0.402073732718894,
#  'comp_novelty': 0.3444700460829493,
#  'novelty': 0.5841013824884793}

In [64]:
os.listdir('./relaxed/relaxed_structures_llamat2_m3gnet_str/')[:5]

['relaxed_2250.cif',
 'relaxed_569.cif',
 'relaxed_6132.cif',
 'relaxed_3167.cif',
 'relaxed_3702.cif']

In [65]:
ffff = ase.io.read('./relaxed/relaxed_structures_llamat2_m3gnet_str/relaxed_2250.cif')

In [66]:
ffff.get_chemical_formula()

'Be4GaRu'

In [67]:
metrics_llamat3_congen = {'comp_valid': 0.6162117452440034,
 'struct_valid': 0.8112489660876757,
 'valid': 0.5062034739454094,
 'comp_div': 15.28758984347853,
 'struct_div': 0.4821768826321802,
 'wdist_density': 4.394579753143331,
 'wdist_num_elems': 0.2793940628797764,
 'cov_recall': 0.9360244648318042,
 'cov_precision': 0.996078431372549,
 'amsd_recall': 0.1957761116921352,
 'amsd_precision': 0.20102959815274782,
 'amcd_recall': 3.761770727043405,
 'amcd_precision': 2.6059243799555727,
 'struc_novelty': 0.9274509803921569,
 'comp_novelty': 0.6186274509803922,
 'novelty': 0.953921568627451,
  'method': 'llamat3-conditional'}


# {'comp_valid': 0.5621181262729125,
#  'struct_valid': 0.6659877800407332,
#  'valid': 0.4256619144602851,
#  'comp_div': 15.04533186738125,
#  'struct_div': 0.5125635516680817,
#  'wdist_density': 11.791204907329718,
#  'wdist_num_elems': 0.31365524639085507,
#  'cov_recall': 0.5012894510622621,
#  'cov_precision': 1.0,
#  'amsd_recall': 0.39510580924426775,
#  'amsd_precision': 0.2147817139006082,
#  'amcd_recall': 7.651830772483437,
#  'amcd_precision': 4.144427428076488,
#  'struc_novelty': 0.9521531100478469,
#  'comp_novelty': 0.8277511961722488,
#  'novelty': 0.9665071770334929}

In [68]:
# metriccs = metrics = {k: v for k,v in metrics.items()}
# metrics['method'] = '17k_14k_cif'
metrics['method'] = 'llamat3_1400_cif'
metrics_llamat3['method'] = 'llamat3_cif'
metrics_llamat2['method'] = 'llamat2_cif';
metrics_llamat3_congen['method'] = 'llamat3_cif_congen'
metrics_llamat2_congen['method'] = 'llamat2_cif_congen'

In [70]:
# metrics

In [72]:
results_df_fn = 'generative_model_results_17k_14k.csv'
results_df = pd.read_csv(results_df_fn)

results_df = pd.concat([
    results_df,
    pd.DataFrame([metrics_llamat2]),
    pd.DataFrame([metrics_llamat3]),
    pd.DataFrame([metrics]),
    pd.DataFrame([metrics_llamat2_congen]),
    pd.DataFrame([metrics_llamat3_congen])
], ignore_index = True)



In [73]:
results_df

,method,struct_valid,comp_valid,cov_recall,cov_precision,wdist_density,wdist_num_elems,valid,comp_div,struct_div,amsd_recall,amsd_precision,amcd_recall,amcd_precision,struc_novelty,comp_novelty,novelty
0,Train,1.000000,0.911300,1.000000,1.000000,0.051000,0.016000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FTCP,0.015500,0.483700,0.047000,0.000900,23.710000,0.736000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GSchNet,0.996500,0.759600,0.383300,0.995700,3.034000,0.641000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PGSchNet,0.775100,0.764000,0.419300,0.997400,4.040000,0.623000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CDVAE,1.000000,0.867000,0.991500,0.994900,0.688000,1.432000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,LM-CH,0.848100,0.835500,0.992500,0.978900,0.864000,0.132000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LM-AC,0.958100,0.888700,0.996000,0.985500,0.696000,0.092000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,llamat2_cif,0.995100,0.878300,0.986369,0.995660,0.622582,0.022799,0.875500,15.537417,0.975478,0.132052,0.114047,3.237658,2.595309,0.397716,0.360708,0.588806
8,llamat3_cif,0.674294,0.602857,0.924721,0.993631,12.355126,0.260724,0.455631,14.813908,0.593143,0.213675,0.221372,4.792127,3.990914,0.915238,0.789319,0.946840
9,llamat3_1400_cif,0.728100,0.650500,0.959352,0.982497,10.404571,0.129352,0.519900,15.049131,0.646088,0.194311,0.216235,4.174398,3.724331,0.872668,0.735718,0.922485


In [36]:
results_df

,method,struct_valid,comp_valid,cov_recall,cov_precision,wdist_density,wdist_num_elems,valid,comp_div,struct_div,amsd_recall,amsd_precision,amcd_recall,amcd_precision,struc_novelty,comp_novelty,novelty
0,Train,1.0000,0.9113,1.0000,1.0000,0.051,0.016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FTCP,0.0155,0.4837,0.0470,0.0009,23.710,0.736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GSchNet,0.9965,0.7596,0.3833,0.9957,3.034,0.641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PGSchNet,0.7751,0.7640,0.4193,0.9974,4.040,0.623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CDVAE,1.0000,0.8670,0.9915,0.9949,0.688,1.432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,LM-CH,0.8481,0.8355,0.9925,0.9789,0.864,0.132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LM-AC,0.9581,0.8887,0.9960,0.9855,0.696,0.092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,llamat2_cif,0.9950,0.8780,0.9860,0.9960,0.623,0.023,0.875,15.537,0.975,0.132,0.114,3.238,2.595,0.398,0.361,0.589
8,llamat3_cif,0.6740,0.6030,0.9250,0.9940,12.355,0.261,0.456,14.814,0.593,0.214,0.221,4.792,3.991,0.915,0.789,0.947
9,llamat3_1400_cif,0.7280,0.6500,0.9590,0.9820,10.405,0.129,0.520,15.049,0.646,0.194,0.216,4.174,3.724,0.873,0.736,0.922


In [85]:
results_df_fn = 'final_results_with_congen.csv'
results_df.to_csv(results_df_fn, index=False)
# results_df_fn = 'llamat2_cif_results.csv'

In [38]:
results_df

,method,struct_valid,comp_valid,cov_recall,cov_precision,wdist_density,wdist_num_elems,valid,comp_div,struct_div,amsd_recall,amsd_precision,amcd_recall,amcd_precision,struc_novelty,comp_novelty,novelty
0,Train,1.0000,0.9113,1.0000,1.0000,0.051,0.016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FTCP,0.0155,0.4837,0.0470,0.0009,23.710,0.736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GSchNet,0.9965,0.7596,0.3833,0.9957,3.034,0.641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PGSchNet,0.7751,0.7640,0.4193,0.9974,4.040,0.623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CDVAE,1.0000,0.8670,0.9915,0.9949,0.688,1.432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,LM-CH,0.8481,0.8355,0.9925,0.9789,0.864,0.132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LM-AC,0.9581,0.8887,0.9960,0.9855,0.696,0.092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,llamat2_cif,0.9950,0.8780,0.9860,0.9960,0.623,0.023,0.875,15.537,0.975,0.132,0.114,3.238,2.595,0.398,0.361,0.589
8,llamat3_cif,0.6740,0.6030,0.9250,0.9940,12.355,0.261,0.456,14.814,0.593,0.214,0.221,4.792,3.991,0.915,0.789,0.947
9,llamat3_1400_cif,0.7280,0.6500,0.9590,0.9820,10.405,0.129,0.520,15.049,0.646,0.194,0.216,4.174,3.724,0.873,0.736,0.922


In [86]:
dfcgenl3 = pd.read_csv('llamat3_cif_adapter_cleaned.csv')
dfcgenl2 = pd.read_csv('llamat2_cif_adapter_cleaned.csv')

In [88]:
dfgentest = pd.read_csv("data/with_tags/test.csv").drop_duplicates().rename(columns={"spacegroup.number": "spacegroup_number"})

In [89]:
dfgentest.head()

,Unnamed: 0,material_id,formation_energy_per_atom,band_gap,pretty_formula,e_above_hull,elements,cif,spacegroup_number
0,6000,mp-10009,-0.575092,0.898,GaTe,0.000000,"['Ga', 'Te']",# generated using pymatgen\ndata_GaTe\n_symmet...,194
1,37702,mp-1218989,-0.942488,0.000,SmThCN,0.044109,"['C', 'N', 'Sm', 'Th']",# generated using pymatgen\ndata_SmThCN\n_symm...,160
2,42245,mp-1225695,0.064863,0.000,CuNi,0.064863,"['Cu', 'Ni']",# generated using pymatgen\ndata_CuNi\n_symmet...,65
3,780,mp-1220884,-1.456116,0.000,NaTiVS4,0.000000,"['Na', 'S', 'Ti', 'V']",# generated using pymatgen\ndata_NaTiVS4\n_sym...,8
4,35749,mp-1224266,0.024139,0.000,Ho3TmMn8,0.036496,"['Ho', 'Mn', 'Tm']",# generated using pymatgen\ndata_Ho3TmMn8\n_sy...,8


In [93]:
ase.io.read('./llamat2_cif_adapter_cleaned/1.cif')

Atoms(symbols='Pr2Ti2O6', pbc=True, cell=[[8.1, 0.0, 0.0], [6.113147599804454, 5.314078134823108, 0.0], [-7.014805770653953, -2.622728407209111, 3.0860647598549114]], spacegroup_kinds=...)

In [94]:
dfgentest[dfgentest.pretty_formula=='Pr2Ti2O6']

,Unnamed: 0,material_id,formation_energy_per_atom,band_gap,pretty_formula,e_above_hull,elements,cif,spacegroup_number


In [ ]:

# def main(args):
    
    

    

    
    
    
    
   
    

    

    metrics = CDVAEGenEval(
        pred_crys, 
        gt_cov_crys,
        gt_novelty_crys,
        n_samples=len(valid_crys), 
        eval_model_name='mp20'
    ).get_metrics()

    metrics = {k: v for k,v in metrics.items()}
    metrics['method'] = args.model_name

    results_df = pd.read_csv(results_df_fn)

    results_df = pd.concat([
        results_df,
        pd.DataFrame([metrics])
    ])

    results_df.to_csv(results_df_fn, index=False)

    print(results_df)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_name", type=str, required=True)
    parser.add_argument("--test_cov_path", type=str, default='data/basic/test.csv')
    parser.add_argument("--test_novelty_path", type=str, default='data/basic/train.csv')
    parser.add_argument("--samples_path", type=str, required=True)
    args = parser.parse_args()

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        main(args)

In [18]:
import pandas as pd

filename="/home/cse/btech/cs1200389/MatLlama/MatLLaMA/src/infer_logs/full_ex_results.csv"
df = pd.read_csv(filename)

for i in range(len(df)): 
    epochs = '1' if '1epochs' in df.iloc[i,0] else '2'
    name = 'LLaMa3' if 'llama3' in df.iloc[i,0] else 'LLama2'# if 'llama2' in df.iloc[i,0]
    name = 'LLaMaT3' if 'llamat3' in df.iloc[i,0] else 'LLaMaT2' if 'llamat2' in df.iloc[i,0] else name
    name += 'chat' if 'chat' in df.iloc[i,0] and 'nochat' not in df.iloc[i,0] else ""
    name = name + "_" + epochs
    df.iloc[i,0] = name

df.to_csv(filename.split('.')[0] + '_namefix.csv', index = False)
print(df)


        file_name  mat_sci_micro_f1  mat_sci_macro_f1  english_micro_f1  \
0    LLaMa3chat_1            77.344            69.767            86.118   
1        LLaMa3_1            85.895            79.106            86.233   
2       LLaMaT3_1            84.584            77.141            84.260   
3    LLama2chat_1            86.601            80.700            83.900   
4        LLama2_1            87.103            81.217            83.830   
5   LLaMaT2chat_1            88.976            83.565            84.542   
6       LLaMaT2_1            88.152            82.324            82.360   
7   LLaMaT3chat_1            89.059            83.183            87.738   
8    LLaMa3chat_2            84.715            78.406            87.145   
9       LLaMaT2_2            91.614            86.414            85.245   
10       LLaMa3_2            88.810            82.742            87.602   
11      LLaMaT3_2            89.611            83.883            86.227   
12       LLama2_2        